In [1]:
%load_ext autoreload
%autoreload 2
!hostname
!pwd
import os, sys
print(sys.executable)

fish-gcp005
/home/akarshkumar0101/nca-alife
/home/akarshkumar0101/miniconda3/envs/nca-alife-torch/bin/python


In [2]:
import os, sys, glob, pickle
from functools import partial

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
# sns.set_theme()

from tqdm.auto import tqdm
from einops import rearrange, reduce, repeat

In [3]:
# import experiment_utils
import time
import subprocess

# Launch Experiment (on this node)

In [9]:
header = '\n'.join(['#!/bin/bash', 'source ~/.virtualenvs/nca-alife-torch/bin/activate', 'cd ~/nca-alife/src'])
gpus = [4, 5, 6, 7]
log_dir = os.path.expanduser(f'~/nca-alife-data/logs/{time.strftime("%Y_%m_%d_%H_%M_%S")}')
exp_sh = f'./src/experiments/main.sh'
with open(exp_sh, 'r') as f:
    commands = [c for c in f.read().split('\n') if c]
print(f"{exp_sh} has {len(commands)} commands: ")
print('\n'.join(commands[:3]), '\n...')
print(f"Logging to {log_dir}")

./src/experiments/main.sh has 24 commands: 
python clip_nca_optim_torch.py --seed=0 --save_dir="/home/akarshkumar0101/nca-alife-data/clip_nca_optim_torch/nca_tgt_0_zeros_zeros_0"    --img_size=32 --substrate="nca" --n_layers=2 --d_state=16 --d_embd=32 --init_state="zeros" --dt=0.01 --p_drop=0.0 --n_steps=64 --target_img_path="/home/akarshkumar0101/nca-alife-data/target_imgs/gorilla.png"       --prompt=None --n_augs=1 --augs="" --bs=1 --lr=0.001 --n_iters=4000 --padding_mode="zeros"   
python clip_nca_optim_torch.py --seed=0 --save_dir="/home/akarshkumar0101/nca-alife-data/clip_nca_optim_torch/nca_tgt_0_zeros_circular_0" --img_size=32 --substrate="nca" --n_layers=2 --d_state=16 --d_embd=32 --init_state="zeros" --dt=0.01 --p_drop=0.0 --n_steps=64 --target_img_path="/home/akarshkumar0101/nca-alife-data/target_imgs/gorilla.png"       --prompt=None --n_augs=1 --augs="" --bs=1 --lr=0.001 --n_iters=4000 --padding_mode="circular"
python clip_nca_optim_torch.py --seed=0 --save_dir="/home/akarsh

In [14]:
print(log_dir)

/home/akarshkumar0101/nca-alife-data/logs/2024_06_17_13_47_35


In [11]:
!mkdir -p {log_dir}
!rm {os.path.dirname(log_dir)+'/latest'}
!ln -sf {log_dir} {os.path.dirname(log_dir)+'/latest'}
for i_gpu, gpu in enumerate(gpus):
    job_ids = np.arange(len(commands))[i_gpu::len(gpus)]
    cvd = f'export CUDA_VISIBLE_DEVICES={gpu}'
    txt = f"{header}\n{cvd}\n\n"
    for job_id in job_ids:
        txt += f"touch {log_dir}/job_{job_id:05d}.status\n"
        txt += f"{commands[job_id]} &> {log_dir}/job_{job_id:05d}.out\n"
        txt += f"echo $? > {log_dir}/job_{job_id:05d}.status\n\n"
    with open(f"{log_dir}/gpu_{gpu}.sh", "w") as f:
        f.write(txt)

In [12]:
print('\n'.join(txt.split('\n')[:12]))

#!/bin/bash
source ~/.virtualenvs/nca-alife-torch/bin/activate
cd ~/nca-alife/src
export CUDA_VISIBLE_DEVICES=7

touch /home/akarshkumar0101/nca-alife-data/logs/2024_06_17_13_47_35/job_00003.status
python clip_nca_optim_torch.py --seed=0 --save_dir="/home/akarshkumar0101/nca-alife-data/clip_nca_optim_torch/nca_tgt_0_point_circular_0" --img_size=32 --substrate="nca" --n_layers=2 --d_state=16 --d_embd=32 --init_state="point" --dt=0.01 --p_drop=0.0 --n_steps=64 --target_img_path="/home/akarshkumar0101/nca-alife-data/target_imgs/gorilla.png"       --prompt=None --n_augs=1 --augs="" --bs=1 --lr=0.001 --n_iters=4000 --padding_mode="circular" &> /home/akarshkumar0101/nca-alife-data/logs/2024_06_17_13_47_35/job_00003.out
echo $? > /home/akarshkumar0101/nca-alife-data/logs/2024_06_17_13_47_35/job_00003.status

touch /home/akarshkumar0101/nca-alife-data/logs/2024_06_17_13_47_35/job_00007.status
python clip_nca_optim_torch.py --seed=0 --save_dir="/home/akarshkumar0101/nca-alife-data/clip_nca_opti

In [13]:
for i_gpu, gpu in enumerate(gpus):
    command = f"bash {log_dir}/gpu_{gpu}.sh"
    print(command)
    process = subprocess.Popen(command, shell=True)

bash /home/akarshkumar0101/nca-alife-data/logs/2024_06_17_13_47_35/gpu_4.sh
bash /home/akarshkumar0101/nca-alife-data/logs/2024_06_17_13_47_35/gpu_5.sh
bash /home/akarshkumar0101/nca-alife-data/logs/2024_06_17_13_47_35/gpu_6.sh
bash /home/akarshkumar0101/nca-alife-data/logs/2024_06_17_13_47_35/gpu_7.sh


# Track Experiment

In [62]:
a = []
for job_id in range(len(commands)):
    try:
        with open(f'{log_dir}/job_{job_id:05d}.status', 'r') as f:
            a.append(int(f.read().strip()))
    except Exception as e:
        print(job_id, e)
        a.append(-1)
a = np.array(a)

17 invalid literal for int() with base 10: ''
19 invalid literal for int() with base 10: ''
21 [Errno 2] No such file or directory: '/home/akarshkumar0101/nca-alife-data/logs/2024_06_17_13_47_35/job_00021.status'
23 [Errno 2] No such file or directory: '/home/akarshkumar0101/nca-alife-data/logs/2024_06_17_13_47_35/job_00023.status'


In [50]:
a

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0, -1,  0,
       -1,  0, -1,  0, -1,  0, -1])

In [51]:
np.array(a).mean()

-0.25

In [61]:
len(commands)

24

In [85]:
for job_id in range(len(commands)):
    print(job_id)
    try:
        with open(f"{log_dir}/job_{job_id:05d}.out", "r") as f:
            print(f.readlines()[-1].strip())
    except Exception as e:
        print(e)

0
100%|██████████| 4000/4000 [02:49<00:00, 23.64it/s, loss=0.000919]
1
100%|██████████| 4000/4000 [06:16<00:00, 10.63it/s, loss=0.0424]
2
100%|██████████| 4000/4000 [03:02<00:00, 21.87it/s, loss=4.53e-5]
3
100%|██████████| 4000/4000 [05:55<00:00, 11.26it/s, loss=0.0054]
4
100%|██████████| 4000/4000 [03:19<00:00, 20.04it/s, loss=0.000327]
5
100%|██████████| 4000/4000 [05:27<00:00, 12.21it/s, loss=0.0424]
6
100%|██████████| 4000/4000 [03:04<00:00, 21.68it/s, loss=0.000175]
7
100%|██████████| 4000/4000 [05:31<00:00, 12.05it/s, loss=0.0521]
8
100%|██████████| 4000/4000 [02:43<00:00, 24.44it/s, loss=0.000344]
9
100%|██████████| 4000/4000 [05:26<00:00, 12.25it/s, loss=0.00558]
10
100%|██████████| 4000/4000 [03:04<00:00, 21.71it/s, loss=0.00104]
11
100%|██████████| 4000/4000 [05:54<00:00, 11.29it/s, loss=0.0521]
12
100%|██████████| 4000/4000 [03:06<00:00, 21.49it/s, loss=0.00026]
13
100%|██████████| 4000/4000 [05:15<00:00, 12.68it/s, loss=0.0367]
14
100%|██████████| 4000/4000 [02:43<00:00, 24